In [1]:
import os 
del os.environ['GDAL_DATA']

In [2]:
import itertools

import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, calculate_default_transform as cdt, Resampling
from rasterstats import zonal_stats
from geocube.api.core import make_geocube
from shapely.geometry import shape, box
from matplotlib.colors import ListedColormap
from matplotlib import cm
import ee
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd

from sepal_ui import mapping as sm

from scripts import parameter as pm
from scripts.utils import update_progress

/usr/local/lib/python3.6/dist-packages/cligj/__init__.py:17: FutureWarning: cligj 1.0.0 will require Python >= 3.7
  warn("cligj 1.0.0 will require Python >= 3.7", FutureWarning)


HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

In [ ]:
ee.Initialize()

Map = sm.SepalMap(['CartoDB.Positron'])

In [ ]:
Map

SepalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

## LMICs

### get the countries borders as ee.FeatureCollection

Get the LMICs countries from the GAUL list and display them on the interactive map

In [ ]:
countries = pm.country_list
countries.head()

,Country,Official name,ISO3,ISO2,UNI,UNDP,FAOSTAT,GAUL
0,Afghanistan,the Islamic Republic of Afghanistan,AFG,AF,4.0,AFG,2.0,1
1,Algeria,the People's Democratic Republic of Algeria,DZA,DZ,12.0,DZA,4.0,4
2,Angola,the Republic of Angola,AGO,AO,24.0,AGO,7.0,8
3,Antigua and Barbuda,Antigua and Barbuda,ATG,AG,28.0,ATG,8.0,11
4,Argentina,the Argentine Republic,ARG,AR,32.0,ARG,9.0,12


In [ ]:
gaul_numbers = countries['GAUL'].tolist()
ee_countries = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.inList('ADM0_CODE', gaul_numbers))
Map.addLayer(ee_countries, {}, 'LMIC')

### transform to shapely

The ee collection need to be transformed into a list of shapely shape to be used as a croping border
need to use a list of polygon because a single polygon exceed the maximum number of vertice

In [ ]:
shapes = []
for index, row in countries.iterrows():
    update_progress(index / (len(countries)-1), 'Countries loaded')
    country = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_CODE', row.GAUL))
    country_json = geemap.ee_to_geojson(country)
    shapes.append(shape(country_json['features'][0]['geometry']))

Countries loaded: [####################] 100.0%


## Croplan map 

Using the map provided by YuanYuan, we will crop the raster on the LMICs map

### cut the map of cropland on the countries

In [ ]:
with rasterio.open(pm.cropland_raster) as src:
    out_image, out_transform = mask(src, shapes, all_touched=True)
    
    out_meta = src.meta.copy()
    out_meta.update(
        driver = 'GTiff',
        height = out_image.shape[1],
        width = out_image.shape[2],
        transform = out_transform,
        compress='lzw'
    )
    with rasterio.open(pm.crop_masked, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
Map.add_raster(pm.crop_masked, layer_name='crop', colormap=cm.get_cmap('YlGn'))

SepalMap(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

## deal with the 2013 LLC zones
from the 2013 map of LLC
Determine the fraction of the cell’s area in the land cover classes in ESA’s CCI-LC maps (10, 20, 30)
each zone will be treated separately to avoid bugs

### functions 

In [ ]:
#transform the raster into a gpd cell grid 
def get_grid(src_rst, max_polygon=0):

    with rasterio.open(src_rst) as src:
        data = src.read(1)

        t = src.transform

        move_x = t[0]
        # t[4] is negative, as raster start upper left 0,0 and goes down
        # later for steps calculation (ymin=...) we use plus instead of minus
        move_y = t[4]

        height = src.height
        width = src.width 

        polygons = []
        indices = list(itertools.product(range(height), range(width)))
        index = 0
        data_flat = data.flatten()
        for y,x in indices:
            if data_flat[index] != np.iinfo(np.int16).min:
                x_min, y_max = t * (x,y)
                x_max = x_min + move_x
                y_min = y_max + move_y
                polygons.append(box(x_min, y_min, x_max, y_max))
            
            index += 1
            
            if (max_polygon != 0) & (len(polygons) > max_polygon):
                break

    return gpd.GeoDataFrame(crs='EPSG:4236', geometry=polygons)

In [ ]:
def gdf_zonal_stats(init_gdf, rst):
    
    gdf = init_gdf.copy()
    gdf = gdf.join(
        pd.DataFrame(
            zonal_stats(
                vectors=gdf['geometry'], 
                raster=rst, 
                all_touched=True, 
                categorical=True
            )
        ),
        how='left'
    )
    
    return gdf

In [ ]:
def fraction_raster(init_gdf, value, template_rst, out_rst):
    
    gdf = init_gdf.copy()
    gdf[f'{value}%'] = gdf[value]/gdf.sum(axis=1)*100
    
    with rasterio.open(template_rst) as src:
        gt = src.transform
        resX = gt[0]
        resY = -gt[4]
        
    cube = make_geocube(
        gdf,
        measurements=[f'{value}%'],
        resolution=(resX, resY),
    )
    
    cube[f'{value}%'].rio.to_raster(out_rst)

In [ ]:
def align_raster(src_rst, template_rst, out_rst):
    
    # get template crs and transform 
    with rasterio.open(template_rst) as tplt, rasterio.open(src_rst) as src:      
        
        kwargs = src.meta.copy()
        kwargs.update(
            driver = 'GTiff',
            height = tplt.height,
            width = tplt.width,
            transform = tplt.transform,
            compress='lzw'
        )
        
        #destination
        with rasterio.open(out_rst, 'w', **kwargs) as dst:
            for i in range(1, dst.count+1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=tplt.transform,
                    dst_crs=tplt.crs,
                    resampling=Resampling.bilinear
                ) 

### get the grid 

This grid will be used as a base for each ecozone value 
sheck the gdf_zonal header to verify that your ecozone is in the polygons regions (not always the case when using max_polygon)

In [ ]:
# value of max polygone
# put to 0 to study the full map
# the 200 first values are placed in the north of kazakstan
max_polygon = 10000

In [ ]:
%%time 
gdf_grid = get_grid(pm.crop_masked, max_polygon=max_polygon)
print(f'{len(gdf_grid)} cells in gdf_grid')

10001 cells in gdf_grid
CPU times: user 963 ms, sys: 209 ms, total: 1.17 s
Wall time: 1.18 s


### Create the 2013 map out of the full map

In [ ]:
#with rasterio.open(pm.llc_full) as src:
#    
#    kwargs = src.meta.copy()
#    kwargs.update(
#        driver = 'GTiff',
#        height = src.height,
#        width = src.width,
#        transform = src.transform,
#        compress='lzw',
#        count = 1
#    )
#    
#    #24 band in total from 1992 to 2015 so 2013 is 22 ( /!\ indexed from 1)
#    data = src.read(22)
#    #print(data.shape)
#    with rasterio.open(pm.llc_2013_raster, 'w', **kwargs) as dst:
#            dst.write(data, indexes=1)

### create the ecozones map

In [ ]:
%%time
gdf_zonal = gdf_zonal_stats(gdf_grid, pm.llc_2013_raster)
gdf_zonal.head()

CPU times: user 1min 46s, sys: 2.07 s, total: 1min 48s
Wall time: 2min 7s


,geometry,210
0,"POLYGON ((-179.91667 89.91667, -179.91667 90.0...",900
1,"POLYGON ((-179.83333 89.91667, -179.83333 90.0...",930
2,"POLYGON ((-179.75000 89.91667, -179.75000 90.0...",930
3,"POLYGON ((-179.66667 89.91667, -179.66667 90.0...",930
4,"POLYGON ((-179.58333 89.91667, -179.58333 90.0...",930


In [ ]:
ecozones = [pm.ecozones[i] for i in pm.ecozones]

for index, zone in enumerate(ecozones):
    
    update_progress(index / (len(ecozones)-1), 'ecozones loaded')
    
    if zone in gdf_zonal.columns:
        fraction_raster(gdf_zonal, zone, pm.crop_masked, pm.llc_2013_map.format(zone))
        align_raster(pm.llc_2013_map.format(zone), pm.crop_masked, pm.llc_2013_map_masked.format(zone))
        Map.add_raster(pm.llc_2013_map_masked.format(zone), layer_name=pm.ecozones[zone])

ecozones loaded: [####################] 100.0%
